In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import missingno as mo
from matplotlib import pyplot as plt

In [ ]:
train = pd.read_csv('../data/train.csv',index_col='Id')

In [ ]:
train.head(5)

### find missing values

In [ ]:
plt.figure(figsize=[12,6],dpi=300)
mo.matrix(train.select_dtypes(include=np.number))
plt.show()

In [ ]:
plt.figure(figsize=[12,6],dpi=300)
mo.matrix(train.select_dtypes(include=np.object))
plt.show()

In [ ]:
# train data
mask = train.isna().sum()
mask = mask[mask>0].sort_values(ascending=False)
mask

In [ ]:
mask.index

In [ ]:
num_feature_miss = train[mask.index].select_dtypes(include=np.number)
cat_feature_miss = train[mask.index].select_dtypes(include=np.object)

In [ ]:
np.mean(num_feature_miss['LotFrontage'])

In [ ]:
# impute lotfrontarea
impute_with = train[['Street','LotFrontage']].groupby(by='Street')['LotFrontage'].mean().to_dict()
impute_with

In [ ]:
replace_with = train.Street.map(impute_with)
train['LotFrontage'] = np.where(train.LotFrontage.isna(),replace_with,train.LotFrontage)

In [ ]:
train[['LotFrontage']].info()

In [ ]:
# impute garage built year
num_feature_miss[['GarageYrBlt']].info()

In [ ]:
train['GarageYrBlt'].fillna(value=0, inplace=True)

In [ ]:
plt.figure(figsize=[24,6],dpi=300)
g = sns.countplot(x=train['GarageYrBlt'],palette='viridis')
g.set_xticklabels(g.get_xticklabels(),rotation=90)
plt.show()

In [ ]:
train['GarageYrBlt'] = train['GarageYrBlt'].astype(int)
train['GarageYrBlt']

In [ ]:
num_feature_miss[['MasVnrArea']].info()

In [ ]:
# impute masvnrarea and msvnrtype
impute_with = train[['MasVnrArea','MasVnrType']].groupby(by='MasVnrType')['MasVnrArea'].mean().to_dict()
impute_with

In [ ]:
train[['HouseStyle','MasVnrType']].groupby(by=['HouseStyle','MasVnrType'])['MasVnrType'].count()/train.shape[0]

In [ ]:
impute_with = {'1.5Fin':'None',
               '1.5Unf':'None',
               '1Story':'Stone',
               '2.5Fin':'None',
               '2.5Unf':'None',
               '2Story':'None',
               'SFoyer':'None',
               'SLvl':'BrkFace'}
replace_with = train.HouseStyle.map(impute_with)
train['MasVnrType'] = np.where(train.MasVnrType.isna(),replace_with,train.MasVnrType)

In [ ]:
train[['MasVnrType']].info()

In [ ]:
impute_with = train[['MasVnrArea','MasVnrType']].groupby(by=['MasVnrType'])['MasVnrArea'].mean().to_dict()
impute_with

In [ ]:
plt.figure(figsize=[12,6],dpi=300)
g = sns.countplot(x=train['MasVnrType'],hue=train['HouseStyle'],palette='viridis')
# g.set_xticklabels(g.get_xticklabels(),rotation=90)
plt.show()

In [ ]:
replace_with = train['MasVnrType'].map(impute_with)
train['MasVnrArea'] = np.where(train.MasVnrArea.isna(),replace_with,train.MasVnrArea)

In [ ]:
train[['MasVnrArea']].info()
